In [ ]:
import pandas as pd
import fastf1 as f1 
import numpy as np 
from timple.timedelta import strftimedelta as td

# Testing 

#["GP, year, session, tyre"]
F1_2022_AUS = f1.get_session(2022, "Australian Grand Prix", "R" )
F1_2022_AUS.load(telemetry=True, laps=True)
drivers_2022_list = F1_2022_AUS.results['Abbreviation'].tolist()

F1_2023_AUS = f1.get_session(2023, 'Australian Grand Prix', 'R')
F1_2023_AUS.load()
drivers_2023_list = F1_2023_AUS.results['Abbreviation'].tolist()
drivers_2023_list.append('RIC')


drivers_list_combined = list(set(drivers_2022_list+drivers_2023_list))
# this is how we will get the fastest time for each 
session = f1.get_session(2022, 'Bahrain Grand Prix','FP1')
session.load(laps=True, telemetry=True, weather=True, messages=False)
datas =[]

# get weather data function 
def get_weather(session):
    session_weather = session.weather_data
    rainCheck = session_weather.loc[session_weather['Rainfall'] == True] 
    if rainCheck.empty:
        return False
    else:
        return True
    

""" weather = get_weather(session)
if weather is not True:
     print( weather )
print(weather) """
# Task: 
# - create a function to get the average of the fastest 3 laps of the session but still keep the fastest lap for the data set 

def avg(session):
        time = session.laps.pick_drivers(i).pick_compounds(info[3]).pick_quicklaps()
        data = np.array(time['LapTime'].nlargest(3))
        avg = (data[0] + data[1] + data[2])/3
          # convert it back into the time m:s.ms
        td = pd.to_timedelta(avg, unit = 'ns')
        print(td)
      
# - create a function to get the gap to the fastest lap from the fastest driver of the session 


def gap(session, compound):
      fastest = session.laps.pick_compounds(compound).pick_fastest()['LapTime']
      gap_time = []
      for i in drivers_list_combined:
        time = session.laps.pick_drivers(i).pick_compounds(compound).pick_fastest()
        if time is not None:
                best = time['LapTime']
                if( best == fastest): 
                        gap_time.append(td(fastest, '%s.%ms'))
                else: 
                        print(best - fastest)
                        gap_time.append('+' + td(best-fastest, '%s.%ms'))
              
        else:
                gap_time.append('N/A')
      return gap_time

  

""" time = session.laps.pick_drivers('HAM').pick_compounds('Hard').pick_fastest()['LapTime']
datas.append(td(time, '%m:%s.%ms'))
print(datas)
 """
dataset = {'Name': drivers_list_combined}
pf = pd.DataFrame(dataset)

array = [] 
gap_data = [] 
index = 'Austrian Grand Prix_2022_SQ_Medium'
index_gap = 'Austrian Grand Prix_2022_SQ_Medium_gap'
info = index_gap.rsplit("_")
session1 = f1.get_session(int(info[1]), info[0], info[2])
session1.load(laps=True, telemetry=True, weather=False, messages=False)  

gap_data = gap(session, info[3])
print(gap_data)
# timedelta object 

for i in drivers_list_combined:
        time1 = session1.laps.pick_drivers(i).pick_compounds(info[3]).pick_fastest()
        if time1 is not None : 
                print('First')
                print(i)
                
        else: 
                print('Seccond')
                print(i)
                array.append('N/A')





""" df = pd.DataFrame(data, columns = [index])

main_df = pd.concat((pf, df), axis = 1)

main_df
 
print(main_df) 
 """




core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 16 completed the race distance 00:00.140000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['16', '11', '63', '44', '4', '3', '31', '77', '10', '23', '24',

0 days 00:00:01.810000
0 days 00:00:01.979000
0 days 00:00:00.493000
0 days 00:00:01.930000
0 days 00:00:00.054000
0 days 00:00:02.247000
0 days 00:00:00.185000
['+ 01.810', 'N/A', 'N/A', '+ 01.979', 'N/A', 'N/A', 'N/A', '+ 00.493', '+ 01.930', 'N/A', 'N/A', 'N/A', 'N/A', '34.557', 'N/A', 'N/A', 'N/A', '+ 00.054', 'N/A', '+ 02.247', 'N/A', '+ 00.185', 'N/A', 'N/A']
First
HAM
Seccond
STR
First
ZHO
First
MSC
Seccond
SAR
First
RIC
First
GAS
First
PER
First
RUS
Seccond
HUL
First
NOR
Seccond
ALB
Seccond
LAT
First
LEC
Seccond
DEV
Seccond
PIA
First
OCO
First
SAI
Seccond
ALO
First
MAG
First
TSU
First
VER
Seccond
VET
First
BOT


' df = pd.DataFrame(data, columns = [index])\n\nmain_df = pd.concat((pf, df), axis = 1)\n\nmain_df\n \nprint(main_df) \n '

In [ ]:
import pandas as pd 
import fastf1 as f1 
from timple.timedelta import strftimedelta as td
import numpy as np

f1.Cache.enable_cache(r'C:\Users\Phuc Trinh\AppData\Local\Temp\fastf1')
f1.set_log_level('WARNING')
# I am choosing the Australian GP randomly because it contains all the main drivers in their main team.  
F1_2022_AUS = f1.get_session(2022, "Australian Grand Prix", "R" )
F1_2022_AUS.load(telemetry=False, laps=True)
drivers_2022_list = F1_2022_AUS.results['Abbreviation'].tolist()

F1_2023_AUS = f1.get_session(2023, 'Australian Grand Prix', 'R')
F1_2023_AUS.load()
drivers_2023_list = F1_2023_AUS.results['Abbreviation'].tolist()
drivers_2023_list.append('RIC')


drivers_list_combined = list(set(drivers_2022_list+drivers_2023_list))
# we are gonna get the list of races for each year as well as 2024 
# we are gonna load in the list of name into the df 

# schedule for 2022 season 
schedule22 = f1.get_event_schedule(2022, include_testing=False, backend = 'ergast')
schedule_2022 = schedule22[ 'EventName'].tolist()

# schedule for 2023 season 
schedule23 = f1.get_event_schedule(2023, include_testing=False, backend = 'ergast')
schedule_2023 = schedule23['EventName'].tolist()

# schedule for 2024 season 
schedule24 = f1.get_event_schedule(2024, include_testing=False, backend = 'ergast')
schedule_2024 = schedule24['EventName'].tolist()



#get columns for data 
session_list_2022 = ['FP1', 'FP2','FP3','Q']
tyre_list = ['Soft', 'Soft_Gap', 'Soft_Avg','Medium', 'Medium_Gap', 'Medium_Avg', 'Hard', 'Hard_Gap', 'Hard_Avg', 'Intermediate', 'Intermediate_Gap', 'Intermediate_Avg','Wet', 'Wet_Gap', 'Wet_Avg']
#columns = []
columns_2022 = []
columns_2023 = [] 
for GP in schedule_2022:
  for session in session_list_2022:
        for tyre in tyre_list:
           if GP == 'Emilia Romagna Grand Prix' or  GP == 'Austrian Grand Prix' or GP == 'São Paulo Grand Prix': 
                if session == 'FP3': break
           else:
            columns_2022.append(GP+'_'+'2022'+'_'+ session +'_'+tyre) 


session_list_2023 = ['FP1', 'FP2','FP3','Q' ,'SS']
tyre_list = ['Soft', 'Soft_Gap', 'Soft_Avg','Medium', 'Medium_Gap', 'Medium_Avg', 'Hard', 'Hard_Gap', 'Hard_Avg', 'Intermediate', 'Intermediate_Gap', 'Intermediate_Avg','Wet', 'Wet_Gap', 'Wet_Avg']
sprint_weekend_2023 =['Azerbaijan Grand Prix', 'Austrian Grand Prix', 'São Paulo Grand Prix' , 'Belgian Grand Prix' ,  'Qatar Grand Prix', ' United States Grand Prix']
for GP in schedule_2023:
  for session in session_list_2023:
        if(GP not in sprint_weekend_2023)  and (session =='SS'):
            continue
        if(GP in sprint_weekend_2023) and (session == 'FP3' or session == 'FP2'): 
            continue  
        for tyre in tyre_list:
           columns_2023.append(GP+'_'+'2023'+'_'+ session +'_'+tyre) 

# Get average lap time from a session and a compound by a driver
def avg(session, compound):
      avg_time = []
      for  i  in drivers_list_combined: 
          time = session.laps.pick_drivers(i).pick_compounds(compound).pick_quicklaps()
          if time.empty is not True: 
            if len(time) == 1: 
              print (time)
              time = session.laps.pick_drivers(i).pick_compounds(compound).pick_fastest()
              if time is not None and time['FastF1Generated'] == False and time['IsAccurate'] == True: 
                avg_time.append(td(time['LapTime'], '%m:%s.%ms'))
              else: 
                avg_time.append('N/A')
            elif len(time) == 2: 
              data = np.array(time['LapTime'].nlargest(2))
              avg = (data[0] + data[1])/2
              # convert it back into the time m:s.ms
              timedelta = pd.to_timedelta(avg, unit = 'ns')
              avg_time.append(td(timedelta, '%m:%s.%ms'))

            elif len(time) >= 3: 
              data = np.array(time['LapTime'].nlargest(3))
              avg = (data[0] + data[1] + data[2])/3
              # convert it back into the time m:s.ms
              timedelta = pd.to_timedelta(avg, unit = 'ns')
              avg_time.append(td(timedelta, '%m:%s.%ms'))
          else: 
              avg_time.append('N/A')
      return avg_time

# get the gap of each driver compared to the fastest lap set during that session using that compound
def gap(session, compound):
      
      gap_time = []
      for i in drivers_list_combined:
        time = session.laps.pick_drivers(i).pick_compounds(compound).pick_fastest()
        if time is not None and time['FastF1Generated'] == False and time['IsAccurate'] == True:
                fastest = session.laps.pick_compounds(compound).pick_fastest()['LapTime']
                best = time['LapTime']
                if( best == fastest): 
                        gap_time.append(td(fastest, '%m:%s.%ms'))
                else: 
                        print(best - fastest)
                        gap_time.append('+' + td(best-fastest, '%s.%ms'))
              
        else:
                gap_time.append('N/A')
      return gap_time

  
def get_data(index):
      # this will split the string passed in and place them into a list. This is how we will get our data.
      # ["GP, year, session, tyre"]
      data = []
      info = index.rsplit("_")
      session = f1.get_session(int(info[1]), info[0], info[2])
      session.load(laps=True, telemetry=False, weather=False, messages=False)  
      if len(info) == 5: 
           # call gap
           if info[4] == 'Gap': 
                data = gap(session, info[3])
           elif info[4] == 'Avg':
                data = avg(session, info[3])
      else:  
        for driver in drivers_list_combined:
          
          time = session.laps.pick_drivers(driver).pick_compounds(info[3]).pick_fastest()
        
          if time is not None and time['FastF1Generated'] == False and time['IsAccurate'] == True : 
                    print('First')
                    data.append( td(time['LapTime'], '%m:%s.%ms')) 
          else: 
                    print('Seccond')
                    data.append('N/A')
                
      print(data)
      return data   




# initialize list of lists
dataset = {'Name': drivers_list_combined}

df_2022 = pd.DataFrame(dataset)
main_2022 = pd.DataFrame()
main_2022 = pd.concat((main_2022, df_2022), axis = 1)
for i in columns_2022:
   print(i)
   try:
    df_current = pd.DataFrame(get_data(i), columns = [i])
    main_2022 = pd.concat((main_2022, df_current), axis = 1) 
   except Exception as ve: 
    print(i)
    print(ve)
    continue
main_2022.to_csv('File2022_updated.csv')

df_2023 = pd.DataFrame(dataset)
main_2023 = pd.DataFrame()
main_2023 = pd.concat((main_2023, df_2023), axis = 1)
for i in columns_2023:
   print(i)
   try:
    df_current = pd.DataFrame(get_data(i), columns = [i])
    main_2023 = pd.concat((main_2023, df_current), axis = 1)
   except Exception as ve: 
    print(i)
    print(ve)
    continue
main_2023.to_csv('File2023_updated.csv')



core        WARNING 	Driver 16 completed the race distance 00:00.140000 before the recorded end of the session.


Bahrain Grand Prix_2022_FP1_Soft
First
Seccond
Seccond
First
First
First
First
Seccond
First
Seccond
Seccond
First
Seccond
First
Seccond
First
First
Seccond
Seccond
First
Seccond
First
First
Seccond
['01:35.923', 'N/A', 'N/A', '01:36.304', '01:35.000', '01:35.028', '01:35.815', 'N/A', '01:34.629', 'N/A', 'N/A', '01:34.814', 'N/A', '01:35.053', 'N/A', '01:34.943', '01:36.402', 'N/A', 'N/A', '01:35.151', 'N/A', '01:34.193', '01:35.644', 'N/A']
Bahrain Grand Prix_2022_FP1_Soft_Gap
0 days 00:00:01.730000
0 days 00:00:02.111000
0 days 00:00:00.807000
0 days 00:00:00.835000
0 days 00:00:01.622000
0 days 00:00:00.436000
0 days 00:00:00.621000
0 days 00:00:00.860000
0 days 00:00:00.750000
0 days 00:00:02.209000
0 days 00:00:00.958000
0 days 00:00:01.451000
['+01.730', 'N/A', 'N/A', '+02.111', '+00.807', '+00.835', '+01.622', 'N/A', '+00.436', 'N/A', 'N/A', '+00.621', 'N/A', '+00.860', 'N/A', '+00.750', '+02.209', 'N/A', 'N/A', '+00.958', 'N/A', '01:34.193', '+01.451', 'N/A']
Bahrain Grand Prix

core        WARNING 	No lap data for driver 22
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 22)


First
Seccond
First
First
Seccond
Seccond
First
Seccond
First
Seccond
Seccond
First
First
First
First
First
First
First
First
Seccond
First
First
First
First
['01:34.868', 'N/A', '01:32.544', '01:33.955', 'N/A', 'N/A', '01:33.971', 'N/A', '01:32.935', 'N/A', 'N/A', '01:33.920', '01:33.437', '01:33.880', '01:32.640', '01:33.121', '01:34.378', '01:32.791', '01:34.295', 'N/A', '01:33.733', '01:34.176', '01:35.667', '01:33.053']
Bahrain Grand Prix_2022_FP3_Soft_Gap


core        WARNING 	No lap data for driver 22
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 22)


...

core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 2)
core        WARNING 	No lap data for driver 2


['N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
Las Vegas Grand Prix_2023_FP1_Wet_Avg


core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 2)


['N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
Las Vegas Grand Prix_2023_FP2_Soft
First
First
First
First
First
First
First
Seccond
First
Seccond
First
First
First
First
First
First
First
First
Seccond
First
First
First
Seccond
First
['01:36.688', '01:36.987', '01:36.183', '01:36.864', '01:35.793', '01:37.412', '01:36.489', 'N/A', '01:36.890', 'N/A', '01:38.140', '01:36.496', '01:36.917', '01:37.656', '01:35.265', '01:36.663', '01:37.680', '01:36.085', 'N/A', '01:37.241', '01:36.129', '01:37.134', 'N/A', '01:35.782']
Las Vegas Grand Prix_2023_FP2_Soft_Gap
0 days 00:00:01.423000
0 days 00:00:01.722000
0 days 00:00:00.918000
0 days 00:00:01.599000
0 days 00:00:00.528000
0 days 00:00:02.147000
0 days 00:00:01.224000
0 days 00:00:01.625000
0 days 00:00:02.875000
0 days 00:00:01.231000
0 days 00:00:01.652000
0 days 00:00:02.391000
0 days 00:00:01.398000
0 days 00:00:02.4

' try: \n    df_2023[i] = get_data(i)\n   except ValueError:\n       print(i)\n       continue '